In [14]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
import matplotlib.pyplot as plt
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
figsize=(12,5)
data_train = pd.read_csv('titanic_data/train.csv')

In [15]:
data_train.loc[(data_train.Age.isnull()),'Age'] = data_train.Age.mean()
def set_Cabin_type(df):
    df.loc[ (df.Cabin.notnull()), 'Cabin' ] = "Yes"
    df.loc[ (df.Cabin.isnull()), 'Cabin' ] = "No"
    return df
data_train = set_Cabin_type(data_train)
dummies_Cabin = pd.get_dummies(data_train['Cabin'], prefix= 'Cabin')

dummies_Embarked = pd.get_dummies(data_train['Embarked'], prefix= 'Embarked')

dummies_Sex = pd.get_dummies(data_train['Sex'], prefix= 'Sex')

dummies_Pclass = pd.get_dummies(data_train['Pclass'], prefix= 'Pclass')

a = data_train.Age
factor=pd.cut(a,[0,15,70,100],labels=['Age_young','Age_adult','Age_old']) #用cut函数来分段向量，在把数据映射到其中，得到0-1向量
b = pd.get_dummies(factor)

df = pd.concat([data_train, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass,b], axis=1)
import sklearn.preprocessing as preprocessing
scaler = preprocessing.StandardScaler()
fare_scale_param = scaler.fit(df['Fare'].values.reshape(-1,1))
df['Fare_scaled'] = scaler.fit_transform(df['Fare'].values.reshape(-1,1), fare_scale_param)
df['AgePclass'] = df['Age']*df['Sex_female']
df.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked','Fare','Age'], axis=1, inplace=True)


In [7]:
from sklearn import linear_model

# 用正则取出我们要的属性值
train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*|AgePclass')
train_np = train_df.values

# y即Survival结果
y = train_np[:, 0]

# X即特征属性值
X = train_np[:, 1:]

# fit到RandomForestRegressor之中
clf = linear_model.LogisticRegression(C=0.5)
clf.fit(X, y)

clf
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
Survived       891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Cabin_Yes      891 non-null uint8
Embarked_C     891 non-null uint8
Embarked_Q     891 non-null uint8
Embarked_S     891 non-null uint8
Sex_female     891 non-null uint8
Sex_male       891 non-null uint8
Pclass_1       891 non-null uint8
Pclass_2       891 non-null uint8
Pclass_3       891 non-null uint8
Age_young      891 non-null uint8
Age_adult      891 non-null uint8
Age_old        891 non-null uint8
Fare_scaled    891 non-null float64
AgePclass      891 non-null float64
dtypes: float64(2), int64(3), uint8(12)
memory usage: 45.3 KB


In [16]:
from sklearn import cross_validation

 #简单看看打分情况
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
all_data = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*|AgePclass')
X = all_data.values[:,1:]
y = all_data.values[:,0]
print(cross_validation.cross_val_score(clf, X, y, cv=5))
all_data

[0.79888268 0.79888268 0.79775281 0.76966292 0.83050847]


,Survived,SibSp,Parch,Cabin_No,Cabin_Yes,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3,Age_young,Age_adult,Age_old,Fare_scaled,AgePclass
0,0,1,0,1,0,0,0,1,0,1,0,0,1,0,1,0,-0.502445,0.000000
1,1,1,0,0,1,1,0,0,1,0,1,0,0,0,1,0,0.786845,38.000000
2,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,0,-0.488854,26.000000
3,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0.420730,35.000000
4,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,-0.486337,0.000000
5,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,-0.478116,0.000000
6,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0,0.395814,0.000000
7,0,3,1,1,0,0,0,1,0,1,0,0,1,1,0,0,-0.224083,0.000000
8,1,0,2,1,0,0,0,1,1,0,0,0,1,0,1,0,-0.424256,27.000000
9,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0,0,-0.042956,14.000000


In [18]:
# 分割数据，按照 训练数据:cv数据 = 7:3的比例
split_train, split_cv = cross_validation.train_test_split(df, test_size=0.3, random_state=0)
train_df = split_train.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*|AgePclass')
# 生成模型
clf = linear_model.LogisticRegression(C=1.0, penalty='l1', tol=1e-6)
clf.fit(train_df.values[:,1:], train_df.values[:,0])

# 对cross validation数据进行预测
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe_titanic = Pipeline([
                        ('clf',LogisticRegression(C=1000.0,penalty='l2',tol=1e-4))])
pipe_titanic.fit(X,y)
pipe_titanic.fit(train_df.values[:,1:], train_df.values[:,0])

cv_df = split_cv.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.|AgePclass')
predictions = pipe_titanic.predict(cv_df.values[:,1:])

origin_data_train = pd.read_csv("Titanic_data/train.csv")
bad_cases = origin_data_train.loc[origin_data_train['PassengerId'].isin(split_cv[predictions != cv_df.values[:,0]]['PassengerId'].values)]
bad_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47 entries, 8 to 882
Data columns (total 12 columns):
PassengerId    47 non-null int64
Survived       47 non-null int64
Pclass         47 non-null int64
Name           47 non-null object
Sex            47 non-null object
Age            36 non-null float64
SibSp          47 non-null int64
Parch          47 non-null int64
Ticket         47 non-null object
Fare           47 non-null float64
Cabin          10 non-null object
Embarked       47 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 4.8+ KB


In [46]:
pd.DataFrame({"columns":list(train_df.columns)[1:], "coef":list(clf.coef_.T)})

,columns,coef
0,SibSp,[-0.4683711735965074]
1,Parch,[-0.17622571266308937]
2,Cabin_No,[0.0]
3,Cabin_Yes,[0.35004865945796954]
4,Embarked_C,[0.0]
5,Embarked_Q,[0.0]
6,Embarked_S,[-0.4472468678328456]
7,Sex_female,[1.5007678370746156]
8,Sex_male,[-0.6815328388483279]
9,Pclass_1,[-0.9992722363095624]


In [19]:
data_test = pd.read_csv("Titanic_data/test.csv")
data_test.loc[ (data_test.Fare.isnull()), 'Fare' ] = 0
# 接着我们对test_data做和train_data中一致的特征变换
# 首先用同样的RandomForestRegressor模型填上丢失的年龄
tmp_df = data_test[['Age','Fare', 'Parch', 'SibSp', 'Pclass']]
data_test.loc[(data_test.Age.isnull()),'Age'] = data_test.Age.mean()

data_test = set_Cabin_type(data_test)
dummies_Cabin = pd.get_dummies(data_test['Cabin'], prefix= 'Cabin')
dummies_Embarked = pd.get_dummies(data_test['Embarked'], prefix= 'Embarked')
dummies_Sex = pd.get_dummies(data_test['Sex'], prefix= 'Sex')
dummies_Pclass = pd.get_dummies(data_test['Pclass'], prefix= 'Pclass')
a = data_test.Age
factor=pd.cut(a,[0,15,70,100],labels=['Age_young','Age_adult','Age_old']) #用cut函数来分段向量，在把数据映射到其中，得到0-1向量
b = pd.get_dummies(factor)
df_test = pd.concat([data_test, dummies_Cabin, dummies_Embarked, dummies_Sex, dummies_Pclass,b], axis=1)
df_test['AgePclass'] = df_test['Age']*df_test['Sex_female']
df_test.drop(['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df_test['Fare_scaled'] = scaler.fit_transform(df_test['Fare'].values.reshape(-1,1), fare_scale_param)
test = df_test.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*|AgePclass')
predictions = pipe_titanic.predict(test)
result = pd.DataFrame({'PassengerId':data_test['PassengerId'].values, 'Survived':predictions.astype(np.int32)})
result.to_csv("Titanic_data/pip.csv", index=False)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 17 columns):
SibSp          418 non-null int64
Parch          418 non-null int64
Cabin_No       418 non-null uint8
Cabin_Yes      418 non-null uint8
Embarked_C     418 non-null uint8
Embarked_Q     418 non-null uint8
Embarked_S     418 non-null uint8
Sex_female     418 non-null uint8
Sex_male       418 non-null uint8
Pclass_1       418 non-null uint8
Pclass_2       418 non-null uint8
Pclass_3       418 non-null uint8
Age_young      418 non-null uint8
Age_adult      418 non-null uint8
Age_old        418 non-null uint8
AgePclass      418 non-null float64
Fare_scaled    418 non-null float64
dtypes: float64(2), int64(2), uint8(13)
memory usage: 18.4 KB
